In [1]:
## RUNNING TIME
import time
import numpy as np
from scipy.integrate import quad
from scipy import stats
import numpy.random as npr
np.random.seed(500)
#Install import_ipynb so we can read dependenies as jupyter notebooks
!pip install ipynb
#Import jupyer notebook depdendencies 
from ipynb.fs.full.FFT_option_valuation_LEWIS import BSM_call_value_FFT, H93_call_value_FFT
from ipynb.fs.full.Lewis_Integration_option_valuation import BSM_call_value, H93_call_value


In [2]:
def BSM_call_valueAnalytical(S0, K, T, r, d, sigma):
    ''' Valuation of European call option in BSM Model.
    Analytical Formula.'''
    d1 = (np.log(S0 / K) + (r-d + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
    d2 = (np.log(S0 / K) + (r-d - 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
    BS_C = (S0*np.exp(-d*T)* stats.norm.cdf(d1, 0.0, 1.0) - K * np.exp(-r * T) * stats.norm.cdf(d2, 0.0, 1.0))
    return BS_C

def MC_Euler_Scheme_Heston_optionpricing(S0, K, T, r, d,
                                         v0, kappa, theta, sigma, rho,
                                         M, I, option = 'put', vt_disct = 'full-truncation'):
    ''' The Monte-Carlo estimation for a European option using the 
    standard Euler-Maruyama scheme in Heston Model'''
    
    ran_num = npr.standard_normal((2, M + 1, I)) 
    dt = T / M
    corr_mat = np.zeros((2, 2))
    corr_mat[0, :] = [1.0, rho]
    corr_mat[1, :] = [rho, 1.0]
    cho_mat = np.linalg.cholesky(corr_mat)
    cho_mat
    v = np.zeros_like(ran_num[0])
    vh = np.zeros_like(v)
    v[0] = v0
    vh[0] = v0
    S = np.zeros_like(ran_num[0])    
    S[0] = S0
    for t in range(1, M + 1):
    
        ran = np.dot(cho_mat, ran_num[:, t, :])
        if vt_disct == 'full-truncation':
        #this scheme is called full-truncation
            vh[t] = (vh[t - 1] + kappa * (theta - np.maximum(vh[t - 1], 0)) * dt
                  + sigma * np.sqrt(np.maximum(vh[t - 1], 0)) * np.sqrt(dt) * ran[1])
            v[t] = np.maximum(0, vh[t])
        
        #this scheme is called Reflection
        elif vt_disct == 'reflection':
            vh[t] = (vh[t - 1] + kappa * (theta - np.abs(vh[t - 1])) * dt
                  + sigma * np.sqrt(np.abs(vh[t - 1])) * np.sqrt(dt) * ran[1])
            v[t] = np.abs(vh[t])
        
        S[t] = S[t - 1] * np.exp((r -d - 0.5 * v[t]) * dt + np.sqrt(v[t]) * ran[0] * np.sqrt(dt))
    if option == 'call':
        hT = np.maximum(S[-1] - K, 0)
    elif option == 'put':
        hT = np.maximum(K - S[-1], 0)

    # calculation of MCS estimator for option price in Heston model
    option_price = np.exp(-(r-d) * T) * np.mean(hT)
    
    return option_price

In [3]:
# Test with sythetic data
# Comparision running time BSM model
# BSM Parameters
S0 = 100.0 ; K = 100.0 ; T = 1.; r = 0.05; sigma = 0.2; d = 0
# Compare running time in one iteration
start_time = time.time()
BSM_call_valueAnalytical(S0, K, T, r, d, sigma)
print("BSM_Analytical method", (time.time() - start_time), "seconds")
start_time = time.time()
BSM_call_value(S0, K, T, r, d, sigma)
print("BSM_FT method", (time.time() - start_time), "seconds")
start_time = time.time()
BSM_call_value_FFT(S0, K, T, r, d, sigma)
print("BSM_FFT method", (time.time() - start_time), "seconds")

# Comparision running time Heston model
# Heston Parameters
S0 = 100.0; K = 100.0; T = 1.0; r = 0.05; d = 0
kappa_v = 2; theta_v = 0.01; sigma_v = 0.1; rho = -0.5; v0 = 0.01

# MC simulation parameters:
M = 252
I = 500000
start_time = time.time()
MC_Euler_Scheme_Heston_optionpricing(S0, K, T, r, d,
                                     v0, kappa_v, theta_v, sigma_v, rho, 
                                     M, I, option = 'call', vt_disct = 'full-truncation')
print("Heston_MC", (time.time() - start_time), "seconds")
start_time = time.time()
H93_call_value(S0, K, T, r, d, kappa_v, theta_v, sigma_v, rho, v0)
print("Heston_Lewis", (time.time() - start_time), "seconds")
start_time = time.time()
H93_call_value_FFT(S0, K, T, r, d, kappa_v, theta_v, sigma_v, rho, v0)
print("Heston_FFT", (time.time() - start_time), "seconds")

BSM_Analytical method 0.0009229183197021484 seconds
BSM_FT method 0.0024831295013427734 seconds
BSM_FFT method 0.002039194107055664 seconds
Heston_MC 18.175643920898438 seconds
Heston_Lewis 0.006893634796142578 seconds
Heston_FFT 0.002832174301147461 seconds


In [4]:
## Test with real data benchmark

import pandas as pd
options = pd.read_csv('option_data.csv')
#Define Stock price (Ericsson B, 28.12.2020)               
S0 = 98.36   
#Dividend yield
d = 0.0086

# Heston Calibrated parameter vector with Lewis:  [2.438 0.116 0.751 -0.416 0.095]
kappa_v = 2.438
theta_v = 0.116
sigma_v = 0.751
rho = -0.416
v0 = 0.095
start_time = time.time()
for row, option in options.iterrows():
    H93_call_value(S0, option['Strike'], option['T'],
                            option['r'], d, kappa_v, theta_v, sigma_v, rho, v0)
H93_call_value_time = (time.time() - start_time)
print("Total time Heston_FT", H93_call_value_time, "seconds")  


# Heston  Calibrated parameter vector with FFT:  [2.181 0.120 0.724 -0.461 0.098]
kappa_v = 2.181
theta_v = 0.120
sigma_v = 0.724
rho = -0.416
v0 = 0.098
start_time = time.time()
for row, option in options.iterrows():    
    H93_call_value_FFT(S0, option['Strike'], option['T'],
                            option['r'], d, kappa_v, theta_v, sigma_v, rho, v0)
    
H93_call_value_FFT_time = (time.time() - start_time)    
print("Total time Heston_FFT", H93_call_value_FFT_time, "seconds")
start_time = time.time()

for row, option in options.iterrows():    
    MC_Euler_Scheme_Heston_optionpricing(S0, option['Strike'], option['T'], option['r'], d,
                                     v0, kappa_v, theta_v, sigma_v, rho, 
                                     M, I, option = 'call', vt_disct = 'full-truncation')
MC_Euler_Scheme_Heston_optionpricing_time = (time.time() - start_time)    
print("Total time Heston_MC", MC_Euler_Scheme_Heston_optionpricing_time, "seconds")    

Total time Heston_FT 0.8336291313171387 seconds
Total time Heston_FFT 0.2584869861602783 seconds
Total time Heston_MC 2343.470215320587 seconds
